In [1]:
#import numpy to use seed 
import numpy as np
import pandas as pd
from numpy.random import seed
np.random.seed(1)

In [2]:
#import other dependencies

import os
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

Read CSV create, and clean dataframe

In [4]:
#Read in CSV and create dataframe
mlb_teams= pd.read_csv("Teams.csv")
mlb_teams = mlb_teams.dropna()
mlb_teams

,yearID,lgID,G,G.1,W,L,WPCT,WSWin,R,AB,...,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,2014.0,NL,162.0,81.0,64.0,98.0,0.57,N,615.0,5552.0,...,47.0,118.0,398.0,1165.0,86.0,4.26,2.0,4.0,35.0,Arizona Diamondbacks
1,2014.0,NL,162.0,81.0,79.0,83.0,0.67,N,573.0,5468.0,...,22.0,123.0,472.0,1369.0,95.0,3.38,5.0,13.0,54.0,Atlanta Braves
2,2014.0,AL,162.0,81.0,96.0,66.0,0.54,N,705.0,5596.0,...,16.0,211.0,401.0,1285.0,44.0,3.43,3.0,13.0,53.0,Baltimore Orioles
3,2014.0,AL,162.0,81.0,71.0,91.0,0.44,N,634.0,5551.0,...,20.0,123.0,535.0,1337.0,63.0,4.01,3.0,7.0,36.0,Boston Red Sox
4,2014.0,NL,162.0,81.0,73.0,89.0,0.42,N,614.0,5508.0,...,31.0,157.0,442.0,1477.0,65.0,3.91,1.0,11.0,37.0,Chicago Cubs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,1969.0,NL,162.0,81.0,88.0,74.0,0.56,N,725.0,5626.0,...,52.0,119.0,454.0,944.0,74.0,3.61,39.0,9.0,33.0,Pittsburgh Pirates
1225,1969.0,NL,162.0,81.0,52.0,110.0,0.46,N,468.0,5357.0,...,42.0,99.0,423.0,1143.0,45.0,4.24,16.0,9.0,25.0,San Diego Padres
1226,1969.0,NL,162.0,81.0,90.0,72.0,0.53,N,713.0,5474.0,...,28.0,136.0,711.0,1054.0,71.0,3.26,71.0,15.0,17.0,San Francisco Giants
1227,1969.0,NL,162.0,80.0,87.0,75.0,0.40,N,595.0,5536.0,...,44.0,90.0,503.0,876.0,87.0,2.94,63.0,12.0,26.0,St. Louis Cardinals


In [5]:
#Get list of dataframe columns

mlb_teams.columns

Index(['yearID', 'lgID', 'G', 'G.1', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H',
       '2B', '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name'],
      dtype='object')

In [6]:
#Select columns for dataframe


mlb_df = mlb_teams[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]
mlb_df.head(20)

,yearID,W,L,WPCT,WSWin,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name
0,2014.0,64.0,98.0,0.57,N,615.0,5552.0,1379.0,259.0,47.0,118.0,398.0,1165.0,86.0,4.26,2.0,4.0,35.0,Arizona Diamondbacks
1,2014.0,79.0,83.0,0.67,N,573.0,5468.0,1316.0,240.0,22.0,123.0,472.0,1369.0,95.0,3.38,5.0,13.0,54.0,Atlanta Braves
2,2014.0,96.0,66.0,0.54,N,705.0,5596.0,1434.0,264.0,16.0,211.0,401.0,1285.0,44.0,3.43,3.0,13.0,53.0,Baltimore Orioles
3,2014.0,71.0,91.0,0.44,N,634.0,5551.0,1355.0,282.0,20.0,123.0,535.0,1337.0,63.0,4.01,3.0,7.0,36.0,Boston Red Sox
4,2014.0,73.0,89.0,0.42,N,614.0,5508.0,1315.0,270.0,31.0,157.0,442.0,1477.0,65.0,3.91,1.0,11.0,37.0,Chicago Cubs
5,2014.0,73.0,89.0,0.56,N,660.0,5543.0,1400.0,279.0,32.0,155.0,417.0,1362.0,85.0,4.29,3.0,6.0,36.0,Chicago White Sox
6,2014.0,76.0,86.0,0.55,N,595.0,5395.0,1282.0,254.0,20.0,131.0,415.0,1252.0,122.0,3.59,5.0,13.0,44.0,Cincinnati Reds
7,2014.0,85.0,77.0,0.39,N,669.0,5575.0,1411.0,284.0,23.0,142.0,504.0,1189.0,104.0,3.56,6.0,15.0,40.0,Cleveland Indians
8,2014.0,66.0,96.0,0.56,N,755.0,5612.0,1551.0,307.0,41.0,186.0,397.0,1281.0,85.0,4.84,1.0,4.0,24.0,Colorado Rockies
9,2014.0,90.0,72.0,0.50,N,757.0,5630.0,1557.0,325.0,26.0,155.0,443.0,1144.0,106.0,4.01,5.0,8.0,41.0,Detroit Tigers


In [7]:
#Use getdummies to convert strings

data_LG = mlb_df.copy()
data_bin = pd.get_dummies(data_LG, columns=["WSWin"])
data_bin = data_bin.drop(columns=['WSWin_N'])

data_bin.head()

,yearID,W,L,WPCT,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,name,WSWin_Y
0,2014.0,64.0,98.0,0.57,615.0,5552.0,1379.0,259.0,47.0,118.0,398.0,1165.0,86.0,4.26,2.0,4.0,35.0,Arizona Diamondbacks,0
1,2014.0,79.0,83.0,0.67,573.0,5468.0,1316.0,240.0,22.0,123.0,472.0,1369.0,95.0,3.38,5.0,13.0,54.0,Atlanta Braves,0
2,2014.0,96.0,66.0,0.54,705.0,5596.0,1434.0,264.0,16.0,211.0,401.0,1285.0,44.0,3.43,3.0,13.0,53.0,Baltimore Orioles,0
3,2014.0,71.0,91.0,0.44,634.0,5551.0,1355.0,282.0,20.0,123.0,535.0,1337.0,63.0,4.01,3.0,7.0,36.0,Boston Red Sox,0
4,2014.0,73.0,89.0,0.42,614.0,5508.0,1315.0,270.0,31.0,157.0,442.0,1477.0,65.0,3.91,1.0,11.0,37.0,Chicago Cubs,0


In [8]:
#Set X(data) and y(team)variables

data = data_bin.drop("name", axis=1)
outcome = data_bin["name"]
print(data.shape, outcome.shape)

(1229, 18) (1229,)


In [9]:
data_bin.count()

yearID     1229
W          1229
L          1229
WPCT       1229
R          1229
AB         1229
H          1229
2B         1229
3B         1229
HR         1229
BB         1229
SO         1229
SB         1229
ERA        1229
CG         1229
SHO        1229
SV         1229
name       1229
WSWin_Y    1229
dtype: int64

Preprocess data

In [10]:
#Import SciKit libraries

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.utils import to_categorical

In [11]:
#Split data and teams into training and test data

X_train, X_test, y_train, y_test = train_test_split(data, outcome, train_size=0.6, test_size=0.4, random_state=42)
X_train

,yearID,W,L,WPCT,R,AB,H,2B,3B,HR,BB,SO,SB,ERA,CG,SHO,SV,WSWin_Y
714,1989.0,81.0,81.0,0.53,632.0,5482.0,1353.0,267.0,30.0,100.0,572.0,958.0,160.0,3.48,20.0,13.0,35.0,0
643,1992.0,72.0,90.0,0.45,574.0,5456.0,1330.0,220.0,36.0,105.0,435.0,1067.0,112.0,3.61,9.0,12.0,30.0,0
845,1984.0,90.0,72.0,0.56,652.0,5438.0,1400.0,235.0,25.0,107.0,500.0,1001.0,149.0,3.60,12.0,15.0,50.0,0
796,1986.0,86.0,75.0,0.60,739.0,5483.0,1386.0,266.0,39.0,154.0,589.0,1154.0,153.0,3.85,22.0,11.0,39.0,0
722,1989.0,73.0,89.0,0.48,694.0,5512.0,1417.0,237.0,29.0,134.0,489.0,838.0,81.0,4.00,15.0,10.0,44.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1044,1976.0,102.0,60.0,0.46,857.0,5702.0,1599.0,271.0,63.0,141.0,681.0,902.0,210.0,3.51,33.0,12.0,45.0,1
1095,1974.0,81.0,81.0,0.43,653.0,5489.0,1441.0,222.0,41.0,110.0,471.0,864.0,108.0,3.46,36.0,18.0,18.0,0
1130,1973.0,60.0,102.0,0.38,548.0,5457.0,1330.0,198.0,26.0,112.0,401.0,966.0,88.0,4.16,34.0,10.0,23.0,0
860,1983.0,71.0,91.0,0.49,701.0,5512.0,1436.0,272.0,42.0,140.0,470.0,868.0,84.0,4.08,9.0,10.0,42.0,0


In [12]:
#Scale data using MinMaxScaler

X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train

714            Montreal Expos
643      San Francisco Giants
845             New York Mets
796     Philadelphia Phillies
722          Seattle Mariners
                ...          
1044          Cincinnati Reds
1095           Houston Astros
1130         San Diego Padres
860              Chicago Cubs
1126         New York Yankees
Name: name, Length: 737, dtype: object

In [13]:
#Label-encode team

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
encoded_y_test

array([12, 21,  6, 18,  7, 14,  7, 27, 13, 10,  0, 20,  1, 18,  5, 28, 13,
        6, 35, 32, 22, 26, 21,  6, 29, 13, 13, 11, 23, 16, 14, 23, 27, 28,
       20,  7, 16, 21, 33,  7, 27, 14, 13, 24, 22, 27, 24, 20, 30,  3, 14,
       27,  7, 17, 24, 19,  4, 16, 27, 21, 21,  3, 18,  2,  3, 18,  2,  5,
       19, 18, 28,  9, 22,  2, 16, 14, 33,  4, 20, 29, 33,  7, 13,  6, 27,
        4,  6, 33, 25, 18, 13,  6, 25, 12, 28, 28,  3, 11, 14, 24, 29, 24,
       26, 26, 25,  2, 21, 28, 19, 22,  4, 34, 16, 28,  6,  3, 22, 31, 31,
        4,  8, 23,  5, 13, 33,  8,  2,  3,  7, 25, 33,  9,  9, 10, 20,  4,
       26, 20, 23,  8,  2, 20, 28, 25, 25, 16, 23, 13, 21, 19, 28, 26, 32,
        6, 32, 18, 25, 23, 23, 13, 32,  7,  5,  3,  3, 14,  2,  4,  6, 33,
        1,  7,  4,  8,  9, 21, 18, 33, 12, 22, 13,  6, 10, 24, 22, 26, 19,
       18,  9, 22,  8, 20, 27, 30, 30, 32,  3,  6, 30, 33, 15, 14, 26,  8,
        7,  8,  1,  1, 23, 11, 32, 22, 16, 24,  9,  4, 25, 10, 25,  9, 23,
       32, 23, 26, 23, 26

In [14]:
#One hot_encode the team data

y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
for x in y_test_categorical:
    print(x)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[

 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 

Create Neural Network Model

In [15]:
# Create model and add layers
model = Sequential()
model.add(Dense(100, input_dim=18, activation='relu'))

model.add(Dense(36, activation='softmax'))

In [16]:
#View model summary

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1900      
_________________________________________________________________
dense_1 (Dense)              (None, 36)                3636      
Total params: 5,536
Trainable params: 5,536
Non-trainable params: 0
_________________________________________________________________


Train Model

In [17]:
#Compile model
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
print(outcome.shape)

(1229,)


In [19]:
#Fit data to train model

model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
24/24 - 0s - loss: 0.1284 - accuracy: 0.0353
Epoch 2/100
24/24 - 0s - loss: 0.1233 - accuracy: 0.0692
Epoch 3/100
24/24 - 0s - loss: 0.1197 - accuracy: 0.1058
Epoch 4/100
24/24 - 0s - loss: 0.1169 - accuracy: 0.1330
Epoch 5/100
24/24 - 0s - loss: 0.1145 - accuracy: 0.1493
Epoch 6/100
24/24 - 0s - loss: 0.1124 - accuracy: 0.1601
Epoch 7/100
24/24 - 0s - loss: 0.1104 - accuracy: 0.1696
Epoch 8/100
24/24 - 0s - loss: 0.1086 - accuracy: 0.1927
Epoch 9/100
24/24 - 0s - loss: 0.1071 - accuracy: 0.2117
Epoch 10/100
24/24 - 0s - loss: 0.1055 - accuracy: 0.2293
Epoch 11/100
24/24 - 0s - loss: 0.1041 - accuracy: 0.2266
Epoch 12/100
24/24 - 0s - loss: 0.1028 - accuracy: 0.2469
Epoch 13/100
24/24 - 0s - loss: 0.1015 - accuracy: 0.2483
Epoch 14/100
24/24 - 0s - loss: 0.1003 - accuracy: 0.2592
Epoch 15/100
24/24 - 0s - loss: 0.0991 - accuracy: 0.2687
Epoch 16/100
24/24 - 0s - loss: 0.0981 - accuracy: 0.2849
Epoch 17/100
24/24 - 0s - loss: 0.0971 - accuracy: 0.2904
Epoch 18/100
24/24 - 0s

Evaluate Model

In [20]:
#Evaluate model

model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

16/16 - 0s - loss: 0.1183 - accuracy: 0.1748
Deep Neural Network - Loss: 0.11832096427679062, Accuracy: 0.17479674518108368


In [21]:
#Read in CSV and create dataframe
mlb_teams2015= pd.read_csv("2015.csv")

mlb2015_df = mlb_teams2015[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]


data2015_LG = mlb2015_df.copy()
data2015_bin = pd.get_dummies(data2015_LG, columns=["WSWin"])
data2015_bin = data2015_bin.drop(columns=['WSWin_N'])

feature2015 = data2015_bin.drop("name", axis=1)
team2015 = data2015_bin["name"]


X_feature2015_scaler = StandardScaler().fit(feature2015)
X_feature2015_scaled = X_feature2015_scaler.transform(feature2015)

label2015_encoder = LabelEncoder()
label2015_encoder.fit(team2015)
encoded_y_2015 = label2015_encoder.transform(team2015)

y_2015_categorical = to_categorical(encoded_y_2015)

encoded_predictions2015 = model.predict_classes(X_feature2015_scaled[:10])
prediction2015_labels = label2015_encoder.inverse_transform(encoded_predictions2015)

print(f"Predicted classes: {prediction2015_labels}")


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


ValueError: y contains previously unseen labels: [32]

In [22]:
#Read in CSV and create dataframe
mlb_teams2016= pd.read_csv("2016.csv")

mlb2016_df = mlb_teams2016[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2016_LG = mlb2016_df.copy()
data2016_bin = pd.get_dummies(data2016_LG, columns=["WSWin"])
data2016_bin = data2016_bin.drop(columns=['WSWin_N'])

feature2016 = data2016_bin.drop("name", axis=1)
team2016 = data2016_bin["name"]

X_feature2016_scaler = StandardScaler().fit(feature2016)
X_feature2016_scaled = X_feature2016_scaler.transform(feature2016)

label2016_encoder = LabelEncoder()
label2016_encoder.fit(team2016)
encoded_y_2016 = label2015_encoder.transform(team2016)

y_2016_categorical = to_categorical(encoded_y_2016)


encoded_predictions2016 = model.predict_classes(X_feature2016_scaled[:10])
prediction2016_labels = label2016_encoder.inverse_transform(encoded_predictions2016)

print(f"Predicted classes: {prediction2016_labels}")

ValueError: y contains previously unseen labels: [30 32]

In [23]:
#Read in CSV and create dataframe
mlb_teams2017= pd.read_csv("2017.csv")

mlb2017_df = mlb_teams2017[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2017_LG = mlb2017_df.copy()
data2017_bin = pd.get_dummies(data2017_LG, columns=["WSWin"])
data2017_bin = data2017_bin.drop(columns=['WSWin_N',])

feature2017 = data2017_bin.drop("name", axis=1)
team2017 = data2017_bin["name"]

X_feature2017_scaler = StandardScaler().fit(feature2017)
X_feature2017_scaled = X_feature2017_scaler.transform(feature2017)

label2017_encoder = LabelEncoder()
label2017_encoder.fit(team2017)
encoded_y_2017 = label2017_encoder.transform(team2017)

y_2017_categorical = to_categorical(encoded_y_2017)




encoded_predictions2017 = model.predict_classes(X_feature2017_scaled[:30])
prediction2017_labels = label2017_encoder.inverse_transform(encoded_predictions2017)

print(f"Predicted classes: {prediction2017_labels}")

ValueError: y contains previously unseen labels: [32]

In [24]:
#Read in CSV and create dataframe
mlb_teams2018= pd.read_csv("2018.csv")

mlb2018_df = mlb_teams2018[['yearID', 'W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2018_LG = mlb2018_df.copy()
data2018_bin = pd.get_dummies(data2018_LG, columns=["WSWin"])
data2018_bin = data2018_bin.drop(columns=['WSWin_N'])

feature2018 = data2018_bin.drop("name", axis=1)
team2018 = data2018_bin["name"]

X_feature2018_scaler = StandardScaler().fit(feature2018)
X_feature2018_scaled = X_feature2018_scaler.transform(feature2018)

label2018_encoder = LabelEncoder()
label2018_encoder.fit(team2018)
encoded_y_2018 = label2018_encoder.transform(team2018)

y_2018_categorical = to_categorical(encoded_y_2018)


encoded_predictions2018 = model.predict_classes(X_feature2018_scaled[:10])
prediction2018_labels = label2018_encoder.inverse_transform(encoded_predictions2018)

print(f"Predicted classes: {prediction2018_labels}")

ValueError: y contains previously unseen labels: [32 33]

In [25]:
#Read in CSV and create dataframe
mlb2019_teams= pd.read_csv("MLB 2019 season.csv")



mlb2019_df = mlb2019_teams[['yearID','W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]


data2019_LG = mlb2019_df.copy()
data2019_bin = pd.get_dummies(data2019_LG, columns=["WSWin"])
data2019_bin = data2019_bin.drop(columns=['WSWin_N'])



feature2019 = data2019_bin.drop("name", axis=1)
team2019 = data2019_bin["name"]


X_feature2019_scaler = StandardScaler().fit(feature2019)
X_feature2019_scaled = X_feature2019_scaler.transform(feature2019)


label_team_encoder2019 = LabelEncoder()
label_team_encoder2019.fit(team2019)
encoded_y_team2019 = label_team_encoder2019.transform(team2019)


y_team2019_categorical = to_categorical(encoded_y_team2019)

encoded_predictions2019 = model.predict_classes(X_feature2019_scaled[:10])
prediction2019_labels = label_team_encoder2019.inverse_transform(encoded_predictions2019)

print(f"Predicted classes: {prediction2019_labels}")


ValueError: y contains previously unseen labels: [32]

In [26]:
#Read in CSV and create dataframe
mlb2020_teams= pd.read_csv("2020.csv")

mlb2020_df = mlb2020_teams[['yearID','W', 'L', 'WPCT', 'WSWin', 'R', 'AB', 'H', '2B',
       '3B', 'HR', 'BB', 'SO', 'SB', 'ERA', 'CG', 'SHO', 'SV', 'name']]

data2020_LG = mlb2020_df.copy()
data2020_bin = pd.get_dummies(data2020_LG, columns=["WSWin"])
#data2020_bin = data2020_bin.drop(columns=['lgID_AL'])

feature2020 = data2020_bin.drop("name", axis=1)
team2020 = data2020_bin["name"]
feature2020


X_feature2020_scaler = StandardScaler().fit(feature2020)
X_feature2020_scaled = X_feature2020_scaler.transform(feature2020)


label_team_encoder2020 = LabelEncoder()
label_team_encoder2020.fit(team2020)
encoded_y_team2020 = label_team_encoder2020.transform(team2020)


y_team2020_categorical = to_categorical(encoded_y_team2020)

encoded_predictions2020 = model.predict_classes(X_feature2020_scaled[:10])
prediction2020_labels = label_team_encoder2020.inverse_transform(encoded_predictions2020)

print(f"Predicted classes: {prediction2020_labels}")


ValueError: y contains previously unseen labels: [33]

Predict World Series Champs

In [28]:
import joblib
filename = 'SWilliams_SVM.sav'
joblib.dump(model, filename)

TypeError: can't pickle _thread.RLock objects